In [1]:
import os
import shutil
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

In [2]:
BASE_DIR = '..'
RANDOM_SEED = 7 # for reproducibility
PROCESSED_DIR = os.path.join(BASE_DIR, 'data', 'processed')

# these relate to training the CNN to predict nightlights
IMAGE_DIR = os.path.join(BASE_DIR, 'data', 'images')

In [3]:
df = pd.read_csv(os.path.join(PROCESSED_DIR, 'finalized_df.csv'))

In [4]:
df.sample(10)

,income_per_cluster,cluster_lat,cluster_long,nightlights,district,dist_lat,dist_long,cluster_name
22,229.227513,32.250834,73.916430,1.434259,gujranwala,32.168056,74.120556,32.250834315384616_73.91642986923078_32.168055...
9,219.954171,32.211450,73.976467,1.383935,gujranwala,32.168056,74.120556,32.21144992307693_73.97646695384616_32.1680555...
111,328.012346,32.093297,74.276652,1.951389,gujranwala,32.168056,74.120556,32.09329674615385_74.27665237692308_32.1680555...
56,274.179012,31.896375,74.096541,1.752130,gujranwala,32.168056,74.120556,31.896374784615386_74.09654112307693_32.168055...
16,224.757425,31.975144,73.916430,1.434259,gujranwala,32.168056,74.120556,31.97514356923077_73.91642986923078_32.1680555...
184,584.185185,31.935759,74.576838,2.706250,gujranwala,32.168056,74.120556,31.935759176923078_74.5768378_32.16805556_74.1...
12,221.955026,32.329603,73.976467,1.383935,gujranwala,32.168056,74.120556,32.3296031_73.97646695384616_32.16805556_74.12...
60,277.659612,31.856990,74.156578,1.844676,gujranwala,32.168056,74.120556,31.856990392307694_74.15657820769232_32.168055...
45,257.462963,32.329603,74.096541,1.752130,gujranwala,32.168056,74.120556,32.3296031_74.09654112307693_32.16805556_74.12...
19,227.731922,32.093297,73.916430,1.434259,gujranwala,32.168056,74.120556,32.09329674615385_73.91642986923078_32.1680555...


Split images into train and validation sets.
The ratio shall be 80 20 => Train Valid => 160 40 

In [17]:
districts = df['district'].unique()

for district in districts:

    filtered_df = df[df['district'] == district].reset_index().drop('index', axis=1)
    X = filtered_df.drop('nightlights', axis=1)  # Assuming 'target_column' is the name of your target variable
    y = filtered_df['nightlights']
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)  # Split into 80% train, 20% val

    print('copying training data for {}'.format(district))
    for im_name, nl in zip(X_train['cluster_name'], y_train):
        src = os.path.join(IMAGE_DIR, district, im_name)
        dest = os.path.join(IMAGE_DIR, district, 'train')
        os.makedirs(dest, exist_ok=True)  # Create destination directory if it doesn't exist
        shutil.copy(src, dest)
    
    print('copying test images for {}'.format(district))
    for im_name, nl in tqdm(zip(X_val['cluster_name'], y_val), total=2):
        src = os.path.join(IMAGE_DIR, district, im_name)
        dest = os.path.join(IMAGE_DIR, district, 'test')
        os.makedirs(dest, exist_ok=True)  # Create destination directory if it doesn't exist
        shutil.copy(src, dest)

copying training data for gujranwala
here !!!
..\data\images\gujranwala\train
..\data\images\gujranwala


FileNotFoundError: [Errno 2] No such file or directory: '..\\data\\images\\gujranwala\\32.17206553076923_74.39672654615386_32.16805556_74.12055556_2.772407407407407.png'

# Model Training - Nightlights Prediction

In [1]:
import os
import copy
import time
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

In [2]:
model_name = 'GAFM_Framework'
model_path = os.path.join('..', 'fine_tuned_models', model_name)
fig_path = os.path.join(model_path, 'figures')
os.makedirs(model_path, exist_ok=True)
os.makedirs(fig_path, exist_ok=True)

In [ ]:
# Define data directory
district = 'gujranwala'
data_dir = os.path.join('..', 'data', 'images', district)
train_dir = os.path.join(data_dir, "train")
valid_dir = os.path.join(data_dir, "valid")

In [ ]:
class AuxiliaryBranch(nn.Module):
    def __init__(self, in_channels, out_channels, downsample=None):
        super(AuxiliaryBranch, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        if self.downsample is not None:
            # Correctly adjust downsample to expect the right input channels
            x = self.downsample(x)
        return x

class GFMM(nn.Module):
    def __init__(self, in_channels_main, in_channels_aux):
        super(GFMM, self).__init__()
        self.batch_norm_main = nn.BatchNorm2d(in_channels_main)
        self.batch_norm_aux = nn.BatchNorm2d(in_channels_aux)
        self.conv_g = nn.Conv2d(in_channels_main + in_channels_aux, in_channels_main, kernel_size=1)
        self.conv_f = nn.Conv2d(in_channels_main + in_channels_aux, in_channels_main, kernel_size=1)
        self.conv_out = nn.Conv2d(in_channels_main, in_channels_main, kernel_size=1)

    def forward(self, fm, fa):
        # Normalize features
        fm_norm = self.batch_norm_main(fm)
        fa_norm = self.batch_norm_aux(fa)
        # Concatenate features
        combined = torch.cat((fm_norm, fa_norm), dim=1)

        # Gating mechanism
        fg = torch.sigmoid(self.conv_g(combined))
        fr = self.conv_f(combined)

        # Output feature
        fo = self.conv_out(fg * fm_norm + (1 - fg) * fr)

        return fo

class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(SEBlock, self).__init__()
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(in_channels, in_channels // reduction, bias=False)
        self.fc2 = nn.Linear(in_channels // reduction, in_channels, bias=False)

    def forward(self, x):
        batch_size, num_channels, _, _ = x.size()
        se = self.global_avg_pool(x).view(batch_size, num_channels)
        se = F.relu(self.fc1(se))
        se = torch.sigmoid(self.fc2(se)).view(batch_size, num_channels, 1, 1)
        return x * se

class SEBottleneckWithGFMM(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, reduction=16):
        super(SEBottleneckWithGFMM, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.se = SEBlock(planes * self.expansion, reduction=reduction)
        self.downsample = downsample
        self.stride = stride

        # Correctly match the number of input channels for downsample_aux
        if stride != 1 or inplanes != planes * self.expansion:
            downsample_aux = nn.Sequential(
                nn.Conv2d(planes * self.expansion, planes * self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * self.expansion),
            )
        else:
            downsample_aux = nn.Sequential(
                nn.Conv2d(planes * self.expansion, planes * self.expansion, kernel_size=1, stride=1, bias=False),
                nn.BatchNorm2d(planes * self.expansion),
            )

        self.aux_branch = AuxiliaryBranch(inplanes, planes * self.expansion, downsample=downsample_aux)
        self.gfmm = GFMM(planes * self.expansion, planes * self.expansion)

    def forward(self, x):
        residual = x
        # Main branch
        main_branch_out = self.conv1(x)
        main_branch_out = self.bn1(main_branch_out)
        main_branch_out = self.relu(main_branch_out)
        main_branch_out = self.conv2(main_branch_out)
        main_branch_out = self.bn2(main_branch_out)
        main_branch_out = self.relu(main_branch_out)
        main_branch_out = self.conv3(main_branch_out)
        main_branch_out = self.bn3(main_branch_out)
        # SE block on the main branch
        main_branch_out = self.se(main_branch_out)
        # Auxiliary branch
        aux_branch_out = self.aux_branch(x)
        # Combine outputs from main and auxiliary branches in GFMM
        combined_out = self.gfmm(main_branch_out, aux_branch_out)
        if self.downsample is not None:
            residual = self.downsample(x)
        # Combine with residual and apply ReLU
        out = combined_out + residual
        out = self.relu(out)
        return out

class SEResNet50WithGFMM(nn.Module):
    def __init__(self, num_classes=1):
        super(SEResNet50WithGFMM, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(SEBottleneckWithGFMM, 64, 3)
        self.layer2 = self._make_layer(SEBottleneckWithGFMM, 128, 4, stride=2)
        self.layer3 = self._make_layer(SEBottleneckWithGFMM, 256, 6, stride=2)
        self.layer4 = self._make_layer(SEBottleneckWithGFMM, 512, 3, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * SEBottleneckWithGFMM.expansion, num_classes)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

In [ ]:
class RegressionDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.file_list = os.listdir(root_dir)

    def __len__(self):
        return len(self.file_list)

    # Inside your RegressionDataset class __getitem__ method
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.file_list[idx])
        image = Image.open(img_name).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = torch.tensor([get_label_from_filename(self.file_list[idx])], dtype=torch.float32)

        return image, label

def get_label_from_filename(filename):
    label_str = filename.split('_')[4][:-4]

    try:
        label = float(label_str)
        return label
    except ValueError:
        print(f"Error extracting label from filename {filename}. Setting label to 0.")
        return 0.0

In [ ]:
# Define transforms for training and validation data
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(299),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
batch_size = 16

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {
    x: RegressionDataset(root_dir=os.path.join(data_dir, x), transform=data_transforms[x])
    for x in tqdm(['train', 'valid'])
}

# Create training and validation dataloaders
dataloaders_dict = {
    x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True)
    for x in tqdm(['train', 'valid'])
}

In [ ]:
# Create the custom ResNet-50 model with SE blocks
model = SEResNet50WithGFMM(num_classes=1)

# Freeze all layers except the last few
for name, param in model.named_parameters():
    if "fc" not in name:
        param.requires_grad = False

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

In [ ]:
def train_model_regression(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = float('inf')
    best_r2 = -float('inf')

    train_losses = []
    train_r2 = []
    valid_losses = []
    valid_r2 = []

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_r2 = 0.0

            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.float().to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    r2 = r2_score(labels.cpu().numpy(), outputs.cpu().detach().numpy())

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_r2 += r2 * inputs.size(0)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_r2 = running_r2 / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} R-squared: {:.4f}'.format(phase, epoch_loss, epoch_r2))

            if phase == 'train':
                train_losses.append(epoch_loss)
                train_r2.append(epoch_r2)
            else:
                valid_losses.append(epoch_loss)
                valid_r2.append(epoch_r2)

            if phase == 'valid' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                best_r2 = epoch_r2

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Loss: {:.4f} Best R-squared: {:.4f}'.format(best_loss, best_r2))

    model.load_state_dict(best_model_wts)

    return model, train_losses, valid_losses, train_r2, valid_r2

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print('device:', device)
model = model.to(device)

In [ ]:
# Train and evaluate for regression
model, train_losses, valid_losses, train_r2, valid_r2 = train_model_regression(model, dataloaders_dict, criterion, optimizer, num_epochs=100)

In [ ]:
# Visualize losses
plt.plot(train_losses, label='Training loss')
plt.plot(valid_losses, label='Validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig(os.path.join(fig_path, 'loss.png'))
plt.show()

# Visualize R-squared values
plt.plot(train_r2, label='Training R-squared')
plt.plot(valid_r2, label='Validation R-squared')
plt.xlabel('Epoch')
plt.ylabel('R-squared')
plt.legend()
plt.savefig(os.path.join(fig_path, 'r2.png'))
plt.show()

In [ ]:
def compute_accuracy_range(r2_scores, mode):
    r2_scores_np = np.array(r2_scores)
    min_r2 = np.min(r2_scores_np)
    max_r2 = np.max(r2_scores_np)
    avg_r2 = np.mean(r2_scores_np)

    print(f"Minimum {mode} R-squared:{min_r2:.2f}")
    print(f"Maximum {mode} R-squared:{max_r2:.2f}")
    print(f"Average {mode} R-squared:{avg_r2:.2f}",'\n')


compute_accuracy_range(train_r2, mode='training')
compute_accuracy_range(valid_r2, mode='testing')

In [ ]:
model_save_path = os.path.join(model_path, model_name + '.pt')
torch.save(model, model_save_path)
print('Model saved successfully !!')